<a href="https://colab.research.google.com/github/addidaswilson/IMDBScraping/blob/main/Web_Scraping_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importing required Libraries
import pandas as pd   #to create dataframe
import requests       #to send the request to the URL
from bs4 import BeautifulSoup #to get the content in the form of HTML
import numpy as np  # to count the values (in our case)
from time import sleep
from random import randint

In [2]:
headers = {"Accept Language": "en-US,en;q=0.5"}

In [3]:

#creating an empty list, so that we can append the values
movie_name = []
movie_genre = []
year = []
time = []
rating = []
metascore = []
votes = []
gross = []
description = []
director= []

In [4]:
pages = np.arange(1,1000,100)

In [5]:
for page in pages:
  #request allow you to send HTTP request
  page = requests.get("https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start="+str(page)+"&ref_=adv_nxt")
  soup = BeautifulSoup(page.content, 'html.parser')
  movie_data = soup.findAll('div', attrs= {'class': 'lister-item mode-advanced'})
  sleep(randint(2,8)) #request will bombard imbd website if too heavy. With this, the request will be passed every 2-8 seconds
  for store in movie_data:
    name = store.h3.a.text
    movie_name.append(name)

    year_of_release = store.h3.find('span',class_='lister-item-year text-muted unbold').text.replace('(', '').replace('I','').replace(')','').replace(' ','')
    year.append(year_of_release)

    length = store.p.find('span',class_='runtime').text.replace('min','')
    time.append(length)

    movie_rating = store.find('div',class_='inline-block ratings-imdb-rating').text.replace('\n','')
    rating.append(movie_rating)

    #score = store.find('span',class_='metascore').text.replace('        ','') if store.find('span',class_='metascore') else '^^^^^^'
    #metascore.append(score)

    value = store.find_all('span', attrs = {'name': 'nv'}) if store.find_all('span', attrs = {'name': 'nv'}) else '^^^^^^'
    vote = value[0].text
    votes.append(vote)

    grosses = value[1].text if len(value) > 1 else '^^^^^^'
    gross.append(grosses)

    desc = store.find_all('p',class_='text-muted')
    description_ = desc[1].text.replace('\n','') if len(desc) > 1 else '^^^^'
    description.append(description_)

    genre_desc = store.find('span',class_='genre').text
    genre = genre_desc.replace('\n','').replace(' ','') 
    movie_genre.append(genre)


    people = store.find_all('p',class_= '')
    people = people[1].text.replace('\n','').replace('Director:','').replace('    ','').split('|')
    director_ = people[0]
    director.append(director_)

In [6]:
#creating a dataframe using pandas library
movie_DF = pd.DataFrame({'Name of movie': movie_name, 'Genre': movie_genre, 'Year of relase': year, 'Watchtime': time, 'Movie Rating': rating, 'Votes': votes, 'Gross Revenue': gross, 'Movie Description': description , 'Director': director})

In [7]:
#Saving data in Excel file:

movie_DF.to_excel("Top_1000_IMDB_Movies.xlsx")

In [8]:
movie_DF

,Name of movie,Genre,Year of relase,Watchtime,Movie Rating,Votes,Gross Revenue,Movie Description,Director
0,Jai Bhim,"Crime,Drama",2021,164,9.4,"163,104",^^^^^^,When a tribal man is arrested for a case of al...,T.J. Gnanavel
1,The Shawshank Redemption,Drama,1994,142,9.3,"2,514,968",$28.34M,Two imprisoned men bond over a number of years...,Frank Darabont
2,The Godfather,"Crime,Drama",1972,175,9.2,"1,732,300",$134.97M,"The Godfather follows Vito Corleone, Don of th...",Francis Ford Coppola
3,Soorarai Pottru,Drama,2020,153,9.1,"105,292",^^^^^^,"Nedumaaran Rajangam ""Maara"" sets out to make t...",Sudha Kongara
4,The Dark Knight,"Action,Crime,Drama",2008,152,9.0,"2,465,196",$534.86M,When the menace known as the Joker wreaks havo...,Christopher Nolan
...,...,...,...,...,...,...,...,...,...
995,Sabrina,"Comedy,Drama,Romance",1954,113,7.6,"62,931",^^^^^^,A playboy becomes interested in the daughter o...,Billy Wilder
996,From Here to Eternity,"Drama,Romance,War",1953,118,7.6,"45,777",$30.50M,"At a U.S. Army base in 1941 Hawaii, a private ...",Fred Zinnemann
997,Gilda,"Drama,Film-Noir,Romance",1946,110,7.6,"31,316",^^^^^^,A small-time gambler hired to work in a Buenos...,Charles Vidor
998,Lifeboat,"Drama,War",1944,97,7.6,"28,147",^^^^^^,Several survivors of a torpedoed merchant ship...,Alfred Hitchcock


# New section